# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,setif,36.1911,5.4137,11.45,47,0,1.54,DZ,1697090108
1,1,zouerate,22.7187,-12.4521,24.92,16,30,3.63,MR,1697090108
2,2,port mathurin,-19.6833,63.4167,23.76,76,46,5.12,MU,1697090108
3,3,baijiantan,45.6333,85.1833,24.00,25,2,3.81,CN,1697090109
4,4,ust-nera,64.5667,143.2000,-7.10,92,100,1.34,RU,1697090109


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5,
    hover_cols = ["City", "Country"]
)
# Display the map
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [37]:
# Narrow down cities that fit criteria and drop any results with null values
city_vacation_df=city_data_df[(city_data_df['Max Temp']>=21) & (city_data_df['Max Temp']<=27) & (city_data_df['Humidity']<=80) & (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
city_vacation_df.dropna()

# Display sample data
city_vacation_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,taroudant,30.4703,-8.8770,25.14,22,0,4.55,MA,1697090110
171,171,changling,44.2653,124.0005,21.69,26,0,2.82,CN,1697090141
183,183,aral,40.5156,81.2636,23.75,18,0,2.05,CN,1697090143
221,221,al kharijah,25.4514,30.5464,24.57,41,0,5.92,EG,1697090150
249,249,bullsbrook,-31.6667,116.0000,26.64,29,0,6.17,AU,1697090155
263,263,broken hill,-31.9500,141.4333,21.60,19,0,12.05,AU,1697090158
306,306,inyati,-19.6756,28.8469,26.30,26,0,5.42,ZW,1697090165
359,359,dakhla,23.6848,-15.9580,22.89,57,0,5.28,EH,1697090175
408,408,yanchep,-31.5500,115.6833,26.68,34,0,4.60,AU,1697090184
448,448,aswan,24.0934,32.9070,23.61,40,0,3.60,EG,1697090192


In [36]:
city_vacation_map = city_vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.5,
    color = "City",
    hover_cols = ["City", "Country","Max Temp"]
)
city_vacation_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp)

### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=city_vacation_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=''

# Display sample data
hotel_df.head(20)

C:\Users\Christophe\AppData\Local\Temp\ipykernel_1160\815083086.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=''


,City,Country,Lat,Lng,Humidity,Hotel Name
11,taroudant,MA,30.4703,-8.8770,22,
171,changling,CN,44.2653,124.0005,26,
183,aral,CN,40.5156,81.2636,18,
221,al kharijah,EG,25.4514,30.5464,41,
249,bullsbrook,AU,-31.6667,116.0000,29,
263,broken hill,AU,-31.9500,141.4333,19,
306,inyati,ZW,-19.6756,28.8469,26,
359,dakhla,EH,23.6848,-15.9580,57,
408,yanchep,AU,-31.5500,115.6833,34,
448,aswan,EG,24.0934,32.9070,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":'accommodation.hotel',
    "limit":1,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=row['Lat']
    lon=row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taroudant - nearest hotel: Hotel Tiouit
changling - nearest hotel: No hotel found
aral - nearest hotel: No hotel found
al kharijah - nearest hotel: Kharga Hotel
bullsbrook - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
inyati - nearest hotel: No hotel found
dakhla - nearest hotel: Hotel El Massira
yanchep - nearest hotel: Yanchep Inn
aswan - nearest hotel: Yaseen hotel
puerto penasco - nearest hotel: Hotel Paraíso Del Desierto
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)


,City,Country,Lat,Lng,Humidity,Hotel Name
11,taroudant,MA,30.4703,-8.8770,22,Hotel Tiouit
171,changling,CN,44.2653,124.0005,26,No hotel found
183,aral,CN,40.5156,81.2636,18,No hotel found
221,al kharijah,EG,25.4514,30.5464,41,Kharga Hotel
249,bullsbrook,AU,-31.6667,116.0000,29,No hotel found
263,broken hill,AU,-31.9500,141.4333,19,Ibis Styles
306,inyati,ZW,-19.6756,28.8469,26,No hotel found
359,dakhla,EH,23.6848,-15.9580,57,Hotel El Massira
408,yanchep,AU,-31.5500,115.6833,34,Yanchep Inn
448,aswan,EG,24.0934,32.9070,40,Yaseen hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
hotel_vacation_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.5,
    color = "City",
    alpha=0.8,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_vacation_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)